In [ ]:
import pandas as pd
import geopandas as gpd
import subprocess
import numpy as np
import shapely.geometry as shpg
from shapely.ops import linemerge
from shapely import set_precision
import os
import sys
import csv
import json
import logging
import shutil
import utm

In [ ]:
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)

In [ ]:
sys.path.append(os.getcwd() + '/../..')
from utils import mkdir, open_zip_shapefile, open_tar_shapefile, haversine, correct_geoms, fix_overaps

In [ ]:
log = logging.getLogger('papermill')
logging.basicConfig(level='INFO', format="%(message)s")

## Files and storage paths

In [ ]:
# Region of interest
reg = 1

# go down from rgi7_scripts/workflow
data_dir = '../../../../rgi7_data/'

# Input dirctory
input_dir = os.path.join(data_dir, 'rgi7_final/regional_files/RGI2000-v7.0-G/')

# Output directories
output_dir = mkdir(os.path.join(data_dir, 'rgi7_rgi6_links'))
output_dir_zip = mkdir(os.path.join(data_dir, 'rgi7_rgi6_links_zip'))

In [ ]:
# Parameters
reg = 1


In [ ]:
reg_str = f'{reg:02d}'

In [ ]:
# RGI v6 file for comparison later 
rgi6_files = {
    '01': '01_rgi60_Alaska.zip',
    '02': '02_rgi60_WesternCanadaUS.zip',
    '03': '03_rgi60_ArcticCanadaNorth.zip',
    '04': '04_rgi60_ArcticCanadaNorth.zip',
    '05': '05_rgi60_GreenlandPeriphery.zip',
    '06': '06_rgi60_Iceland.zip',
    '07': '07_rgi60_Svalbard.zip',
    '08': '07_rgi60_Scandinavia.zip',
    '09': '09_rgi60_RussianArctic.zip',
    '10': '10_rgi60_NorthAsia.zip',
    '11': '11_rgi60_CentralEurope.zip',
    '12': '12_rgi60_CaucasusMiddleEast.zip',
    '13': '13_rgi60_CentralAsia.zip',
    '14': '14_rgi60_SouthAsiaWest.zip',
    '15': '15_rgi60_SouthAsiaEast.zip',
    '16': '16_rgi60_LowLatitudes.zip',
    '17': '17_rgi60_SouthernAndes.zip',
    '18': '18_rgi60_NewZealand.zip',
    '19': '19_rgi60_AntarcticSubantarctic.zip',
}
rgi6_reg_file = os.path.join(data_dir, 'l0_RGIv6', rgi6_files[reg_str])

### Load the input data

In [ ]:
reg_file = os.path.join(data_dir, '00_rgi70_regions', '00_rgi70_O1Regions')
reg_file = gpd.read_file(reg_file)
reg_file = reg_file.loc[reg_file.o1region == f'{reg:02d}'].iloc[0]

input_regfile = f'{input_dir}/RGI2000-v7.0-G-{reg_file.long_code}.zip'

In [ ]:
# Read files
shp = open_zip_shapefile(input_regfile)

## Links to RGI6

In [ ]:
import overlaps_helpers

In [ ]:
# load RGI6 data
from utils import open_zip_shapefile
rgi6 = open_zip_shapefile(rgi6_reg_file)

In [ ]:
# Compute RGI7 - RGI6 overlaps
overlaps = overlaps_helpers.compute_cross_overlaps(shp.geometry, rgi6.geometry)

In [ ]:
# Add more stats
overlaps['area_km2'] = overlaps['geometry'].to_crs({'proj':'cea'}).area * 1e-6
overlaps['i'] = shp['rgi_id'].iloc[overlaps['i']].values
overlaps['j'] = rgi6['RGIId'].iloc[overlaps['j']].values

In [ ]:
# Filter by minimum area. See https://github.com/ezwelty/rgi_links/issues/6
# print(len(overlaps))
overlaps = overlaps[overlaps['area_km2'] > (200 * 1e-6)].copy()
# print(len(overlaps))

In [ ]:
# Count number of direct relatives (i.e. 1:1, n:1, 1:n, n:n)
overlaps['in'], overlaps['jn'] = overlaps_helpers.count_pair_relations(
  overlaps['i'], overlaps['j']
)
# Label clusters of (directly and indirectly-related) pairs
overlaps['cluster'] = overlaps_helpers.label_pair_clusters(overlaps['i'], overlaps['j'])

In [ ]:
# Remove geometry for now
odf_links = overlaps[['i', 'j', 'area_km2', 'i_area_fraction', 'j_area_fraction', 'cluster', 'in', 'jn']].copy()
odf_links.columns = ['rgi7_id', 'rgi6_id', 'overlap_area_km2', 'rgi7_area_fraction', 'rgi6_area_fraction', 'cluster_id', 'n_rgi7', 'n_rgi6']

In [ ]:
odf_links;

### Write out

In [ ]:
print('Writing...')
odf_links.to_csv(output_dir + f'/RGI2000-v7.0-G-{reg_file.long_code}-rgi6_links.csv', quoting=csv.QUOTE_NONNUMERIC, index=False)